In [22]:
%pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.


In [23]:
import cv2
import mediapipe as mp 
import numpy as np 
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [24]:
help(mp_drawing.DrawingSpec)

Help on class DrawingSpec in module mediapipe.python.solutions.drawing_utils:

class DrawingSpec(builtins.object)
 |  DrawingSpec(color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2) -> None
 |  
 |  DrawingSpec(color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2)
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __init__(self, color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2) -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  -------------------------------------------------

In [25]:
from IPython.display import Image

Image(url="https://i.imgur.com/3j8BPdc.png", width=300)

In [26]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

[x: 0.49964866
y: 0.69485676
z: -1.1424255
visibility: 0.99865603
, x: 0.5269658
y: 0.63751495
z: -1.0756671
visibility: 0.9975793
, x: 0.5417106
y: 0.6377827
z: -1.0759481
visibility: 0.9980008
, x: 0.5559428
y: 0.6399976
z: -1.0763203
visibility: 0.9973482
, x: 0.47068205
y: 0.64693534
z: -1.0777445
visibility: 0.9975706
, x: 0.4521846
y: 0.65239704
z: -1.0771471
visibility: 0.9982181
, x: 0.43515882
y: 0.6593534
z: -1.0777879
visibility: 0.9981139
, x: 0.5856976
y: 0.6776289
z: -0.6003878
visibility: 0.99797624
, x: 0.41933164
y: 0.6923541
z: -0.5933391
visibility: 0.99873525
, x: 0.53987104
y: 0.7599592
z: -0.9603902
visibility: 0.99883515
, x: 0.46809047
y: 0.76683056
z: -0.95936406
visibility: 0.99906677
, x: 0.746702
y: 1.0042499
z: -0.28372025
visibility: 0.98172677
, x: 0.29495484
y: 0.9844602
z: -0.33494675
visibility: 0.9951934
, x: 0.8730717
y: 1.3354046
z: -0.34098303
visibility: 0.07273316
, x: 0.14303648
y: 1.1220851
z: -0.43361002
visibility: 0.3328789
, x: 0.85484487
y

In [27]:
len(landmarks)

33

In [28]:
for lndmrk in mp_pose.PoseLandmark: 
    print(lndmrk)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [29]:
landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

x: 0.3866768
y: 1.7982552
z: 0.10375333
visibility: 0.0026402196

In [30]:
landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]

x: 0.33613467
y: 2.0936503
z: -0.444426
visibility: 0.0112450775

In [31]:
landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]

x: 0.4349443
y: 2.6311312
z: -0.14885505
visibility: 0.00075223704

In [32]:
def calculate_squat_angle(hip, knee, ankle):
    hip = np.array(hip) # First
    knee = np.array(knee) # Mid
    ankle = np.array(ankle) # End
    
    radians = np.arctan2(ankle[1]-knee[1], ankle[0]-knee[0]) - np.arctan2(hip[1]-knee[1], hip[0]-knee[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle


In [33]:
hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]


In [34]:
hip, knee, ankle 

([0.3866767883300781, 1.798255205154419],
 [0.336134672164917, 2.0936503410339355],
 [0.4349443018436432, 2.631131172180176])

In [35]:
calculate_squat_angle(hip, knee, ankle)

159.8738552814719

In [36]:
tuple(np.multiply(hip, [640, 480]).astype(int))
tuple(np.multiply(knee, [640, 480]).astype(int))
tuple(np.multiply(ankle, [640, 480]).astype(int))

(278, 1262)

All lines from next to camera is squat form logic


In [37]:
landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]

x: 0.28960925
y: 0.9877479
z: -0.31485564
visibility: 0.9520567

In [38]:
landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

x: 0.3866768
y: 1.7982552
z: 0.10375333
visibility: 0.0026402196

In [40]:
landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]

x: 0.33613467
y: 2.0936503
z: -0.444426
visibility: 0.0112450775

In [41]:
def calculate_torso_angle(right_shoulder, right_hip, right_knee):
    right_shoulder = np.array(right_shoulder)  
    right_hip = np.array(right_hip)  
    right_knee = np.array(right_knee)  
    
    # Calculate the angle between hip, knee, and shoulder
    radians = np.arctan2(right_knee[1] - right_hip[1], right_knee[0] - right_hip[0]) - np.arctan2(right_shoulder[1] - right_hip[1], right_shoulder[0] - right_hip[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    
    return angle


In [42]:
right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
right_hip= [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
right_knee= [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]

In [43]:
right_shoulder, right_hip, right_knee

([0.28960925340652466, 0.9877479076385498],
 [0.3866767883300781, 1.798255205154419],
 [0.336134672164917, 2.0936503410339355])

In [44]:
calculate_torso_angle(right_shoulder, right_hip, right_knee)

163.4614096184458

In [45]:
# Convert landmark coordinates to pixel values
tuple(np.multiply(right_shoulder, [640, 480]).astype(int))
tuple(np.multiply(right_hip, [640, 480]).astype(int))
tuple(np.multiply(right_knee, [640, 480]).astype(int))


(215, 1004)

In [48]:
cap = cv2.VideoCapture(0)

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for squat landmarks
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_hip= [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee= [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            
            # Calculate squat angle
            squat_angle = calculate_squat_angle(hip, knee, ankle)
            torso_angle = calculate_torso_angle(right_shoulder, right_hip, right_knee)
            
            # Visualize angle
            cv2.putText(image, str(squat_angle), 
                           tuple(np.multiply(knee, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            cv2.putText(image, str(torso_angle), 
                           tuple(np.multiply(right_hip, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
                       
        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Squat Checker', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [59]:
cap = cv2.VideoCapture(0)

# Squat counter variables
counter = 0 
stage = None
feedback = ""

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for squat landmarks
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_hip= [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee= [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            
            # Calculate squat angle
            squat_angle = calculate_squat_angle(hip, knee, ankle)
            torso_angle = calculate_torso_angle(right_shoulder, right_hip, right_knee)
            
            
            # Visualize angle
            cv2.putText(image, str(squat_angle), 
                           tuple(np.multiply(knee, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(torso_angle), 
                           tuple(np.multiply(right_hip, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Squat counter logic
            if squat_angle < 95:
                stage = "down"
            if stage == "down":
                if torso_angle < 55 or torso_angle > 80:
                    feedback = "Maintain a neutral spine & maintain 60-85 degree hip" 
                elif torso_angle > 55 or torso_angle < 80:
                    feedback = "That is good form keep it up"
            if squat_angle > 170 and stage =="down":
                stage="up"
                counter +=1
                print(counter)
                       
        except:
            pass
        
        # Render squat counter
        # Setup status box
        cv2.rectangle(image, (0, 0), (640, 100), (245, 117, 16), -1)

        
        # Text for 'SQUATS'
        cv2.putText(image, 'SQUATS', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2, cv2.LINE_AA)

        # Text for 'STAGE'
        cv2.putText(image, 'STAGE', (165, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, (160, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2, cv2.LINE_AA)

        # Text for 'FORM' (Ensure it fits within the box)
        feedback_lines = feedback.split('\n')
        if len(feedback_lines) > 0:
            feedback_line1 = feedback_lines[0]
        else:
            feedback_line1 = ''

        if len(feedback_lines) > 1:
            feedback_line2 = feedback_lines[1]
        else:
            feedback_line2 = ''

        cv2.putText(image, 'FORM', (290, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, feedback_line1, (270, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, feedback_line2, (270, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Squat Tracker', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    
    cv2.destroyAllWindows()


1
2
3
4
